# Ch3 LLM, RAG 개인 필수과제

### Goal: LLM과 RAG 기술을 활용해 사용자 질문에 답변하는 챗봇

##### 1. 사용환경 준비

In [ ]:
!pip install langchain langchain-openai langchain-community faiss-cpu
!pip install -q pypdf

In [ ]:
import os
from getpass import getpass

os.environ["KMP_DUPLICATE_LIB_OK"] = "True"  # 라이브러리끼리의 충돌을 해결
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ")

##### 2. 모델 로드하기

In [3]:
from langchain_openai import ChatOpenAI

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

##### 3. 문서 로드하기

In [4]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("files/인공지능산업최신동향_2024년_11월호.pdf")

# 페이지 별 문서 로드
docs = loader.load()

##### 4. 문서 청크로 나누기

**CharacterTextSplitter**: 주어진 텍스트를 문자 단위로 분할하는 데 사용하는 클래스

- `separator`: 분할된 각 청크를 구분할 때 기준이 되는 문자열
- `chunk_size`: 각 청크의 최대 길이
- `chunk_overlap`: 인접한 청크 사이에 중복으로 포함될 문자의 수
- `length_function`: 청크의 길이를 계산하는 함수
- `is_separator_regex`: 구분자로 정규식을 사용할지 여부를 설정

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = text_splitter.split_documents(docs)

for i, chunk in enumerate(splits):
    print(f"id: {i+1} {chunk}", end="\n\n")
    
    # 청킹된 내용을 상위 50개까지 출력
    if i+1 == 50:
        break

**RecursiveCharacterTextSplitter**
- 주어진 문자 목록의 순서대로 청크가 충분히 작아질 때까지 재귀적으로 텍스트를 분할하는 클래스
- 문자 목록을 매개변수로 받아 동작
- 기본 문자 목록: \["\n\n", "\n", " ", ""\] (default)
- 텍스트를 재귀적으로 분할하여 의미적으로 관련있는 텍스트 조각들이 같이 있도록 하는 목적으로 설계됨

---
- `chunk_size`: 각 청크의 최대 길이
- `chunk_overlap`: 인접한 청크 사이에 중복으로 포함될 문자의 수
- `length_function`: 청크의 길이를 계산하는 함수
- `is_separator_regex`: 구분자로 정규식을 사용할지 여부를 설정

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(docs)

for i, chunk in enumerate(splits):
    print(f"id: {i+1} {chunk}", end="\n\n")
    
    # 청킹된 내용을 상위 50개까지 출력
    if i+1 == 50:
        break

##### 5. 벡터 임베딩 생성

In [ ]:
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

##### 6. 벡터 스토어 생성

In [8]:
from langchain_community.vectorstores import FAISS


vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

##### 7. FAISS를 Retriever로 변환

In [9]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})

##### 8. 프롬프트 템플릿 정의

In [10]:
from langchain_core.prompts import ChatPromptTemplate

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

##### 9. RAG 체인 구성

In [11]:
from langchain_core.runnables import RunnablePassthrough

class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output
# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# RAG 체인에서 각 단계마다 DebugPassThrough 추가
rag_chain_debug = {
    "context": retriever,                    # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()        # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
}  | DebugPassThrough() | ContextToText() | contextual_prompt | model


##### 10. 챗봇 구동 확인

In [36]:
i=0
while i<3:
	print("========================")
	query = input("질문을 입력하세요: ")
	response = rag_chain_debug.invoke(query)
	print("Final Response:")
	print(response.content)
	i += 1

Debug Output: 일본 AI 안전연구소에서 발간한 AI 안정성 평가 가이드에 대해 요약해줘.
Debug Output: {'context': [Document(metadata={'source': 'files/인공지능산업최신동향_2024년_11월호.pdf', 'page': 16}, page_content='일본 AI안전연구소, AI 안전성에 대한 평가 관점 가이드 발간n일본 AI안전연구소는 AI 개발자나 제공자가 안전성 평가에 참조할 수 있는 ‘AI 안전성에 대한 평가 관점')], 'question': '일본 AI 안전연구소에서 발간한 AI 안정성 평가 가이드에 대해 요약해줘.'}
Final Response:
일본 AI안전연구소는 AI 개발자와 제공자가 AI 안전성 평가에 참고할 수 있는 가이드를 발간했습니다. 이 가이드는 AI의 안전성을 평가하는 다양한 관점을 제시하여, AI 시스템의 신뢰성과 안정성을 높이는 데 도움을 주고자 합니다.
Debug Output: 인공지능 산업에 대한 미국의 정책들을 요약해서 설명해줘
Debug Output: {'context': [Document(metadata={'source': 'files/인공지능산업최신동향_2024년_11월호.pdf', 'page': 1}, page_content='Ⅰ. 인공지능 산업 동향 브리프 1. 정책/법제    ▹ 미국 민권위원회, 연방정부의 얼굴인식 기술 사용에 따른 민권 영향 분석························1')], 'question': '인공지능 산업에 대한 미국의 정책들을 요약해서 설명해줘'}
Final Response:
미국의 인공지능 산업에 대한 정책들은 주로 민권과 관련된 이슈를 다루고 있습니다. 특히, 미국 민권위원회는 연방정부의 얼굴인식 기술 사용이 개인의 민권에 미치는 영향을 분석하고 있습니다. 이는 인공지능 기술의 사용이 개인의 자유와 권리에 어떤 영향을 미치는지를 평가하려는 노력을 반영합니다.
Debug Output: 한국 기업인 카카오에서 제공하는 AI 

In [39]:
# 일반 ChatGPT와 답변 비교
llm = ChatOpenAI(model="gpt-4o-mini")

for i in range(3):
    print("========================")
    query = input("질문을 입력하세요: ")
    print(f"query: {query}")
    response = llm.invoke(query)
    print("Final Response:")
    print(response.content)

query: 일본 AI 안전연구소에서 발간한 AI 안정성 평가 가이드에 대해 요약해줘.
Final Response:
일본 AI 안전연구소에서 발간한 AI 안정성 평가 가이드는 인공지능 시스템의 안전성과 신뢰성을 확보하기 위한 지침을 제공합니다. 이 가이드는 다음과 같은 주요 내용을 포함하고 있습니다:

1. **안전성 기준**: AI 시스템이 충족해야 할 기본적인 안전성 기준과 요구사항을 정의합니다. 이는 시스템의 설계, 개발, 운영 과정에서 고려해야 할 요소들입니다.

2. **위험 평가**: AI 시스템이 실제 환경에서 발생할 수 있는 위험 요소를 식별하고 평가하는 방법론을 제시합니다. 이를 통해 잠재적인 위험을 사전에 인지하고 대응할 수 있도록 합니다.

3. **테스트 및 검증**: AI 시스템의 안전성을 확보하기 위해 필요한 테스트 및 검증 절차를 설명합니다. 이는 다양한 시나리오와 조건에서 시스템의 성능을 검토하는 데 중점을 둡니다.

4. **모니터링 및 피드백**: AI 시스템이 운영되는 동안 지속적으로 모니터링하고, 발생하는 문제에 대한 피드백을 통해 개선할 수 있는 방법을 제안합니다.

5. **법적 및 윤리적 고려사항**: AI 시스템의 안전성을 평가하는 과정에서 법적, 윤리적 측면도 고려해야 한다는 점을 강조합니다. 이는 사회적 신뢰를 구축하는 데 중요한 요소입니다.

이 가이드는 AI 기술이 점점 더 많은 분야에 적용됨에 따라, 그 안전성과 신뢰성을 확보하는 데 기여하고자 하는 목적을 가지고 있습니다.
query: 인공지능 산업에 대한 미국의 정책들을 요약해서 설명해줘
Final Response:
미국의 인공지능(AI) 산업에 대한 정책은 여러 가지 주요 방향성과 목표를 가지고 있습니다. 다음은 이러한 정책들을 요약한 내용입니다.

1. **AI 연구 및 개발 지원**: 미국 정부는 AI 기술의 연구와 개발을 촉진하기 위해 대규모 자금을 지원하고 있습니다. 이를 통해 AI의 혁신을 가속화하고, 경쟁력을 유지하고자 합니다. 예를 들어, 국

***RAG VS 일반 GPT***

1. 정보 접근
- 일반 GPT 모델
    - 사전에 학습된 데이터에 의존하여 대답
    - 사전 학습 데이터 이후의 정보 혹은 학습된 지식을 뛰어 넘는 정보에 대한 질문은 답변하기 어려움
- RAG 모델
    - 외부 검색 시스템이나 데이터베이스로부터 <span style="color:lime">필요한 정보를 검색</span>하여 응답 생성에 활용
    - 일반 GPT 모델보다 최신 정보를 제공하기에 알맞음

2. 데이터 출처의 신뢰성
- 일반 GPT 모델
    - 학습된 정보를 바탕으로 추론을 하므로, 정보의 출처를 명확히 제시할 수 없고 특정한 사실에 대해 신뢰도가 떨어질 수 있음
- RAG 모델
    - 필요한 정보를 검색하는 과정을 거치므로, 정보의 출처를 확인할 수 있고 검색된 자료의 신뢰도를 판단할 수 있음

3. 학습된 지식 업데이트
- 일반 GPT 모델
    - 모델을 다시 훈련하지 않는 이상 지식이 업데이트 되지 않음
- RAG 모델
    - 새로운 정보를 검색함으로써 지식의 업데이트 없이도 실시간 데이터에 접근 가능

4. 요약
- **최신 정보나 특정 지식을 필요로 하는 경우, 한정된 지식으로 인해 생길 수 있는 문제를 해소하고 보다 신뢰성 높은 정보를 제공하기 위해 RAG 모델이 필요**
- RAG 모델은 카카오의 신규 AI 서비스인 카나나에 대해 잘 설명한 반면, 일반 모델은 그렇지 못함